# Make a table of Absorption lines

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from astropy.table import Table, vstack
from matplotlib import pyplot as plt

from pyigm.igm import igmsightline
from cgmsquared import io

import yaml

from astropy.io import fits
from astropy.wcs import WCS
import astropy.units as u
from astropy.coordinates import SkyCoord

In [2]:
data_path = "/Users/mwilde/Dropbox/IGM_spectra/Completed_QSOs/"
abs_file = 'J0809+4619_werksquad.json'

### Read in the json file

In [3]:
qso = igmsightline.IGMSightline.from_json(data_path+abs_file)

read_sets: Using set file -- 
  /Users/mwilde/linetools/linetools/lists/sets/llist_v1.2.ascii
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12
AbsComp coordinates do not match.  Best to set them
AbsComp coordinates do not match.  Best to set them
AbsComp coordinates do not match.  Best to set them


### Make the igmsystems

Use the component list to generate a list of IGMSystems

In [4]:
abssys = qso.make_igmsystems()

### Since these are raw werksquad files, we need to artificially set the error and the flag to get a column density

In [5]:
ion_species = []
for a in abssys:
    for c in a._components:
        # comp = abssys[0]._components[0]
        for absline in c._abslines:
            absline.attrib['flag_N'] = 1
            absline.attrib['sig_N'] = np.array([1e13,1e13])/u.cm**2
        c.synthesize_colm(overwrite=True)
    a.update_component_colm(overwrite=True)
    
    # Fills the ionN Table from the list of components
    a.fill_ionN()
    ion_species.append(a._ionN)
    


### Make a table of all the absorption systems

In [6]:
abssysTable = vstack([ion_species[i]for i in range(len(ion_species))])
abssysTable

RA,DEC,comp_name,Z,ion,Ej,vmin,vmax,ion_name,flag_N,logN,sig_logN [2],b,sig_b,vel,sig_vel,comment,reliability,z_sys
deg,deg,,,,1 / cm,km / s,km / s,,,,,km / s,km / s,km / s,km / s,,,
float64,float64,str15,int64,int64,float64,float64,float64,str5,int64,float64,float64,float64,float64,float64,float64,str51,str1,float64
122.2839,46.3238,z-0.00023_OI,8,1,0.0,-42.65575012791966,42.65575012791082,OI,1,14.64,0.009949111803676972 .. 0.009949111803676972,0.0,0.0,0.0,0.0,,a,-0.00023045480505031152
122.2839,46.3238,z-0.00018_SiIII,14,3,0.0,-127.14694749670068,127.14694749607868,SiIII,1,14.11,0.03371198393451511 .. 0.03371198393451511,0.0,0.0,0.0,0.0,,a,-0.00023045480505031152
122.2839,46.3238,z-0.00015_CII,6,2,0.0,-123.8657359484213,123.86573594784647,CII,1,15.77,0.00073753784719136 .. 0.00073753784719136,0.0,0.0,0.0,0.0,,a,-0.00023045480505031152
122.2839,46.3238,z-0.00013_FeII,26,2,0.0,-92.69422623981966,92.69422623956984,FeII,1,15.04,0.0016169950339052527 .. 0.0016169950339052527,0.0,0.0,0.0,0.0,,a,-0.00023045480505031152
122.2839,46.3238,z-0.00013_SiIV,14,4,0.0,-77.9287742722147,77.92877427211964,SiIV,1,13.51,0.09490067762745767 .. 0.09490067762745767,0.0,0.0,0.0,0.0,,a,-0.00023045480505031152
122.2839,46.3238,z-0.00010_CIV,6,4,0.0,-80.38968293335732,80.38968293342103,CIV,1,14.06,0.026746644999065664 .. 0.026746644999065664,0.0,0.0,0.0,0.0,,a,-0.00023045480505031152
122.2839,46.3238,z-0.00010_SiII,14,2,0.0,-117.30331285181728,117.30331285181211,SiII,1,14.98,0.002033758163159027 .. 0.002033758163159027,0.0,0.0,0.0,0.0,,a,-0.00023045480505031152
122.2839,46.3238,z-0.00008_SII,16,2,0.0,-70.54604828849811,70.54604828849995,SII,1,15.51,0.0007748607881054352 .. 0.0007748607881054352,0.0,0.0,0.0,0.0,,a,-0.00023045480505031152


In [7]:
group_by_z = abssysTable.group_by('z_sys')
group_by_z

RA,DEC,comp_name,Z,ion,Ej,vmin,vmax,ion_name,flag_N,logN,sig_logN [2],b,sig_b,vel,sig_vel,comment,reliability,z_sys
deg,deg,,,,1 / cm,km / s,km / s,,,,,km / s,km / s,km / s,km / s,,,
float64,float64,str15,int64,int64,float64,float64,float64,str5,int64,float64,float64,float64,float64,float64,float64,str51,str1,float64
122.2839,46.3238,z-0.00023_OI,8,1,0.0,-42.65575012791966,42.65575012791082,OI,1,14.64,0.009949111803676972 .. 0.009949111803676972,0.0,0.0,0.0,0.0,,a,-0.00023045480505031152
122.2839,46.3238,z-0.00018_SiIII,14,3,0.0,-127.14694749670068,127.14694749607868,SiIII,1,14.11,0.03371198393451511 .. 0.03371198393451511,0.0,0.0,0.0,0.0,,a,-0.00023045480505031152
122.2839,46.3238,z-0.00015_CII,6,2,0.0,-123.8657359484213,123.86573594784647,CII,1,15.77,0.00073753784719136 .. 0.00073753784719136,0.0,0.0,0.0,0.0,,a,-0.00023045480505031152
122.2839,46.3238,z-0.00013_FeII,26,2,0.0,-92.69422623981966,92.69422623956984,FeII,1,15.04,0.0016169950339052527 .. 0.0016169950339052527,0.0,0.0,0.0,0.0,,a,-0.00023045480505031152
122.2839,46.3238,z-0.00013_SiIV,14,4,0.0,-77.9287742722147,77.92877427211964,SiIV,1,13.51,0.09490067762745767 .. 0.09490067762745767,0.0,0.0,0.0,0.0,,a,-0.00023045480505031152
122.2839,46.3238,z-0.00010_CIV,6,4,0.0,-80.38968293335732,80.38968293342103,CIV,1,14.06,0.026746644999065664 .. 0.026746644999065664,0.0,0.0,0.0,0.0,,a,-0.00023045480505031152
122.2839,46.3238,z-0.00010_SiII,14,2,0.0,-117.30331285181728,117.30331285181211,SiII,1,14.98,0.002033758163159027 .. 0.002033758163159027,0.0,0.0,0.0,0.0,,a,-0.00023045480505031152
122.2839,46.3238,z-0.00008_SII,16,2,0.0,-70.54604828849811,70.54604828849995,SII,1,15.51,0.0007748607881054352 .. 0.0007748607881054352,0.0,0.0,0.0,0.0,,a,-0.00023045480505031152


In [8]:
print(abssysTable['ion_name', 'logN', 'z_sys'])

ion_name  logN          z_sys         
-------- ----- -----------------------
      OI 14.64 -0.00023045480505031152
   SiIII 14.11 -0.00023045480505031152
     CII 15.77 -0.00023045480505031152
    FeII 15.04 -0.00023045480505031152
    SiIV 13.51 -0.00023045480505031152
     CIV 14.06 -0.00023045480505031152
    SiII 14.98 -0.00023045480505031152
     SII 15.51 -0.00023045480505031152
    AlII 14.08 -0.00023045480505031152
      NI 14.78 -0.00023045480505031152
     ...   ...                     ...
      HI  14.6      0.6562107040329883
     NIV 13.74      0.6562107040329883
    CIII 13.34      0.6562107040329883
     OIV 14.51      0.6562107040329883
     OVI 15.37      0.6562107040329883
    CIII 13.89      0.6562107040329883
     SVI 13.18      0.6562107040329883
      HI 14.74      0.6562107040329883
     OIV 14.49      0.6562107040329883
     NIV 13.79      0.6562107040329883
      HI 13.63       0.657625273920309
Length = 123 rows


# or load it from cgmsquared

In [9]:
import cgmsquared

In [10]:
cgmsquared.utils.make_abssys_table(data_path+abs_file)

Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12
AbsComp coordinates do not match.  Best to set them
AbsComp coordinates do not match.  Best to set them
AbsComp coordinates do not match.  Best to set them


RA,DEC,comp_name,Z,ion,Ej,vmin,vmax,ion_name,flag_N,logN,sig_logN [2],b,sig_b,vel,sig_vel,comment,reliability,z_sys
deg,deg,,,,1 / cm,km / s,km / s,,,,,km / s,km / s,km / s,km / s,,,
float64,float64,str15,int64,int64,float64,float64,float64,str5,int64,float64,float64,float64,float64,float64,float64,str51,str1,float64
122.2839,46.3238,z-0.00023_OI,8,1,0.0,-42.65575012791966,42.65575012791082,OI,1,14.64,0.009949111803676972 .. 0.009949111803676972,0.0,0.0,0.0,0.0,,a,-0.00023045480505031152
122.2839,46.3238,z-0.00018_SiIII,14,3,0.0,-127.14694749670068,127.14694749607868,SiIII,1,14.11,0.03371198393451511 .. 0.03371198393451511,0.0,0.0,0.0,0.0,,a,-0.00023045480505031152
122.2839,46.3238,z-0.00015_CII,6,2,0.0,-123.8657359484213,123.86573594784647,CII,1,15.77,0.00073753784719136 .. 0.00073753784719136,0.0,0.0,0.0,0.0,,a,-0.00023045480505031152
122.2839,46.3238,z-0.00013_FeII,26,2,0.0,-92.69422623981966,92.69422623956984,FeII,1,15.04,0.0016169950339052527 .. 0.0016169950339052527,0.0,0.0,0.0,0.0,,a,-0.00023045480505031152
122.2839,46.3238,z-0.00013_SiIV,14,4,0.0,-77.9287742722147,77.92877427211964,SiIV,1,13.51,0.09490067762745767 .. 0.09490067762745767,0.0,0.0,0.0,0.0,,a,-0.00023045480505031152
122.2839,46.3238,z-0.00010_CIV,6,4,0.0,-80.38968293335732,80.38968293342103,CIV,1,14.06,0.026746644999065664 .. 0.026746644999065664,0.0,0.0,0.0,0.0,,a,-0.00023045480505031152
122.2839,46.3238,z-0.00010_SiII,14,2,0.0,-117.30331285181728,117.30331285181211,SiII,1,14.98,0.002033758163159027 .. 0.002033758163159027,0.0,0.0,0.0,0.0,,a,-0.00023045480505031152
122.2839,46.3238,z-0.00008_SII,16,2,0.0,-70.54604828849811,70.54604828849995,SII,1,15.51,0.0007748607881054352 .. 0.0007748607881054352,0.0,0.0,0.0,0.0,,a,-0.00023045480505031152
